In [ ]:
#Generate Energy List
import forgi.threedee.model.coarse_grain as ftmc
import os

def get_rmsd_dict(rmsd_list):
    rmsd_dict = {}
    with open(rmsd_list, "r") as fh:
        for line in fh.readlines():
            name, rmsd = (line.rstrip()).split("\t")
            rmsd_dict[name] = float(rmsd)
    return rmsd_dict

rmsd_dict = get_rmsd_dict("../data/train_rmsd_list.txt")

file_path = "../data/train_set/" #"../data/old_training_set/" # "../data/val_set/"#"../data/test_set/" #

energy_file = "../data/train_energy.txt" #"../data/old_train_energy.txt"#"../data/val_energy.txt"#"../data/test_energy.txt" #

write = False

#file = "../data/train_set/1c2x_C_5srRNA_C_br0.cg"
c_files = 0
c_has = 0
c_not = 0
pos_e = 0
lines = []
for file in [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f))]:
    if file in rmsd_dict.keys():
        cg = ftmc.CoarseGrainRNA.from_bg_file(file_path + file)
        cg.radius_of_gyration
        energy = cg.infos['Energy']
    
        c_files += 1
        if energy == []:
            c_not += 1
            print(file, energy, cg.radius_of_gyration(method="vres"), rmsd_dict[file])
        elif "ROG" in energy[0]:
            c_has +=1
            rog_en = energy[0].split(",")[0][:-4]
            if float(rog_en) > 0:
                #print(file, rog_en)
                pos_e +=1
            line = file + "\t" + rog_en + "\n"
            lines.append(line)

    

print("files", c_files)
print("ROG energy", c_has)
print("Positive energy", pos_e)
print("no energy", c_not)
print(c_has + c_not)

if write:
    with open(energy_file, "w") as fh:
        for l in lines:
            fh.write(l)


In [2]:
#Rotate cg structures
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os

directory = "../data/old_training_set/"
out_dir = "../data/5st_test_rot/"
angle = 1.5708


for file in [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]:

    cgf = ftmc.CoarseGrainRNA.from_bg_file(directory + file)
    cgf.rotate(angle)

    with fuc.open_for_out(out_dir + file) as outfile:
        print(cgf.to_cg_string(), file=outfile)

In [6]:
#Transform coords to vectors, pointing to the next connected element
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os
import numpy as np

cg = ftmc.CoarseGrainRNA.from_bg_file("../data/train_set/1c2x_C_5srRNA_C_be10.cg")

cg_d = dict(cg.coords)

#for e in cg_d:
#    print(e, cg_d[e])

print(cg.coords.get_direction("t0"))

dist = cg.element_physical_distance("t0", "m0")

print(dist)
for elem in cg_d:
    vector = []
    if elem == "t0" or elem == "f0":
        for i in range(3):
            a = cg_d[elem][0][i] - cg_d[elem][1][i]
            vector.append(a)
    else:
        for i in range(3):
            a = cg_d[elem][1][i] - cg_d[elem][0][i]
            vector.append(a)

    print(elem, np.array(vector))

[ 9.6719918  -3.48100864 -1.15317169]
17.849
s0 [ 0.    17.849  0.   ]
m0 [ 9.1820551  10.94019783  3.16790342]
s1 [-0.06612564 -2.56114239  0.16020625]
i0 [ 0.40567371 -5.40125577  1.58513772]
s2 [ 0.67507931 -5.06854354  0.08444098]
i1 [ 3.56446688  6.38357025 12.75647898]
s3 [-6.01970619 -1.85064955  4.36224193]
h0 [-5.1620225  25.22911748 -9.45248275]
m1 [ 1.15852993  5.37241537 -0.38377324]
s4 [ 1.02590916  2.29657981 -0.51256276]
i2 [-1.72501499 17.34190355  8.31710371]
s5 [ 7.17459534  3.50431681 15.96345033]
h1 [ 2.89447963 -8.16404602  9.34800693]
m2 [-10.34058503 -16.31261319  -2.78413018]
t0 [-9.6719918   3.48100864  1.15317169]


In [1]:
#get nearest k elements
#--> set of k vectors
#plus origin? ie middle of element
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import os
import numpy as np

cg = ftmc.CoarseGrainRNA.from_bg_file("../data/train_set/1c2x_C_5srRNA_C_be10.cg")

cg_d = dict(cg.coords)

#calculate midpoint of each element
mp_dir = {}
for elem in cg_d:
    mp = (cg_d[elem][0] + cg_d[elem][1])/2
    mp_dir[elem] = mp

for e in mp_dir:
    print(e, mp_dir[e])

#calculate distance from each midpoint to every other
dist_dir = {}
for a in mp_dir:
    helper_d = {}
    for b in mp_dir:
        if a != b:
            dist = np.linalg.norm(mp_dir[b] - mp_dir[a])
            helper_d[b] = dist
    if helper_d != {}:
        dist_dir[a] = helper_d

#get the nearest k=5 elements
n_dict = {}
for f in dist_dir:
    n_list = []
    #print(f, {k: v for k, v in sorted(dist_dir[f].items(), key=lambda item: item[1])})
    i = 0
    for n in  {k: v for k, v in sorted(dist_dir[f].items(), key=lambda item: item[1])}:
        n_list.append(n)
        i+=1
        if i == 3:
            break
    n_dict[f] = n_list

print(n_dict)

for elem in n_dict:
    v_arr = []
    for e in n_dict[elem]:
        v_arr.append(mp_dir[e])
    print(elem, np.concatenate(v_arr))

s0 [0.     8.9245 1.    ]
m0 [ 4.59102755 23.31909891  2.58395171]
s1 [ 9.14899228 27.50862663  4.24800654]
i0 [ 9.31876632 23.52742755  5.12067852]
s2 [ 9.85914283 18.2925279   5.95546788]
i1 [11.97891593 18.95004125 12.37592786]
s3 [10.75129627 21.21650161 20.93528831]
h0 [ 5.16043193 32.90573557 18.3901679 ]
m1 [ 9.76132007 31.47540551  3.9760168 ]
s4 [10.85353961 35.3099031   3.5278488 ]
i2 [10.50398669 45.12914478  7.43011927]
s5 [13.22877687 55.55225496 19.57039629]
h1 [18.26331436 53.22239035 32.22612492]
m2 [ 5.17029252 26.0053066   2.39206509]
t0 [ 4.8359959  -1.74050432  0.42341416]
{'s0': ['t0', 's2', 'm0'], 'm0': ['m2', 'i0', 's1'], 's1': ['m1', 'i0', 'm2'], 'i0': ['s1', 's2', 'm0'], 's2': ['i0', 'i1', 'm0'], 'i1': ['s2', 's3', 'i0'], 's3': ['i1', 'h0', 's2'], 'h0': ['s3', 'm1', 's1'], 'm1': ['s4', 's1', 'm2'], 's4': ['m1', 's1', 'i2'], 'i2': ['s4', 'm1', 's5'], 's5': ['h1', 'i2', 'h0'], 'h1': ['s5', 'i2', 'h0'], 'm2': ['m0', 's1', 'i0'], 't0': ['s0', 's2', 'i1']}
s0 [ 4.83

In [6]:
#Open lmdb files
import lmdb
import gzip
import json
import numpy as np
import pandas as pd
#import pdbtools

'''
b'id_to_idx'
b'num_examples'
b'serialization_format'
'''


lmdb_file = "/home/milan/Programs/ares_release/data/lmdbs/train/"
lmdb_env = lmdb.open(lmdb_file)
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()

print(lmdb_env.stat())

idx = lmdb_cursor.get("idx_to_idx".encode())#.decode() #gzip.decompress(
print(idx)


i = gzip.decompress(lmdb_cursor.get("0".encode())).decode()

d = json.loads(i)

i_json = pd.read_json(i)

columns = i_json.iloc[0,0]
index = i_json.iloc[1,0]
data = i_json.iloc[2,0]

i_new = pd.DataFrame(data=data, index=index, columns=columns)

pdb_out = pd.DataFrame(i_new, columns=["serial_number", "subunit", "chain", "residue", "resname", "x", "y", "z", "element", "name"])

pdb_out.head()

{'psize': 4096, 'depth': 1, 'branch_pages': 0, 'leaf_pages': 1, 'overflow_pages': 256, 'entries': 53}
None


,serial_number,subunit,chain,residue,resname,x,y,z,element,name
0,1,0,A,1,C,7.278,9.150,14.138,C,C5'
1,2,0,A,1,C,6.457,10.282,13.572,C,C4'
2,3,0,A,1,C,7.273,11.210,12.812,O,O4'
3,4,0,A,1,C,5.459,9.762,12.567,C,C3'
4,5,0,A,1,C,4.329,9.250,13.226,O,O3'


In [10]:
#get list of representative RNA structures

rep_pdbs = []
with open("../data/nrlist_3.220_4.0A.csv", "r") as fh:
    for line in fh.readlines():
        equ = line.split(",")[0].lower().lstrip('"').rstrip('"')
        all = line.split(",")[1].lower().lstrip('"').rstrip('"')
        if "+" in all:
            l = all.split("+")
            for i in l:
                id = i.split("|")
                rep_pdbs.append(id+[equ])
        else:
            pdb = all.split("|")
            rep_pdbs.append(pdb+[equ])

rfam_pdbs = {}
with open("../data/Rfam.pdb", "r") as fh:
    for line in fh.readlines()[1:]:
        pdb = line.split("\t")
        rfam_pdbs[pdb[1]] = pdb


c = 0
nc = 0
lines = []
rmc_dict = {}
for item in rep_pdbs:
    if item[0] in rfam_pdbs.keys():
        c +=1
    else:
        nc +=1
    l = item[0] + "\t" + item[1] + "\t" + item[2] + "\t" + item[3] + "\n"
    lines.append(l)

    

with open("../data/rep_rfam_list.txt", "w") as fh:
    for line in lines:
        fh.write(line)

print(rep_pdbs[0])
print("len rep pdbs", len(rep_pdbs))
print("len rfam pdbs", len(rfam_pdbs))
print("rep in rfam", c)
print("rep not in rfam", nc)

#for k in sorted(rmc_dict):
#    print(k, rmc_dict[k])


['4v9q', '1', 'dv', 'nr_4.0_29909.1']
len rep pdbs 2862
len rfam pdbs 2305
rep in rfam 656
rep not in rfam 2206


In [36]:
rep_pdbs = []
with open("../data/nrlist_3.220_4.0A.csv", "r") as fh:
    for line in fh.readlines():
        all = line.split(",")[1].lower().lstrip('"').rstrip('"')
        if "+" in all:
            l = all.split("+")
            for i in l:
                id = i.split("|")
                rep_pdbs.append(id)
        else:
            pdb = all.split("|")
            rep_pdbs.append(pdb)

print(len(rep_pdbs))
with open("../data/reprna_list.txt", "w") as fh:
    for n in rep_pdbs:
        fh.write("\t".join(n) + "\n")

2862


In [15]:
id_dict= {}
with open("../../rep_rna_list.csv", "r") as fh:
    for line in fh.readlines()[1:]:
        num, equ, rep = line.split(",")
        
        id = equ.split(";")[0]
        rna_id = rep.split(";")[1]
        
        if rna_id not in id_dict.keys():
            id_dict[rna_id] = [id]
        else:
            id_dict[rna_id].append(id)

for key in id_dict:
    print(key, id_dict[key])

  RNA (5'-R(*(GCP)P*G)-3') ['NR_4.0_96460.1']
  RNA (5'-R(*GP*UP*C)-D(P*U)-R(P*CP*CP*UP*[...] ['NR_4.0_84831.1']
  RNA (5'-D(*(ATP))-R(P*A)-3') ['NR_4.0_60897.1']
  RNA (5'-D(*(GTG))-R(P*A)-3') ['NR_4.0_36733.1']
  5'-R(*CP*(5AA)*(HFA))-3' ['NR_4.0_35535.1']
  RNA (5'-R(*(6MZ)P*C)-3') ['NR_4.0_33222.1', 'NR_4.0_19822.1', 'NR_4.0_63313.1', 'NR_4.0_10291.1']
  m7G-capped RNA ['NR_4.0_23607.1']
  O2'methyl-RNA ['NR_4.0_01981.1']
  RNA (5'-R(P*GP*G)-3') ['NR_4.0_98428.1']
  RNA (5'-R(P*CP*U)-3') ['NR_4.0_96254.1']
  RNA (5'-R(P*AP*G)-3') ['NR_4.0_94833.3']
  RNA 3-mer (de novo synthesized) ['NR_4.0_93339.3']
  RNA (5'-R(P*AP*GP*G)-3') ['NR_4.0_80196.1']
  DNA (5'-R(*GP*U)-3') ['NR_4.0_78372.4']
  RNA (5'-D(*(APC))-R(P*GP*U)-3') ['NR_4.0_77810.2']
  RNA (5'-D(*(ATP))-R(P*AP*A)-3') ['NR_4.0_73502.1']
  DNA/RNA (5'-D(*IP*)-R(*CP*)-D(*IP*)-R(*C[...] ['NR_4.0_70203.1']
  RNA (5'-R(P*AP*GP*CP*A)-3') ['NR_4.0_68310.1']
  5'-R(UP*A)-3') ['NR_4.0_65802.1']
  RNA/DNA chimera (5'-D(CP*CP*)R(UP*UP*)D(

In [6]:
#is it possible to get elements interacting in 3d space?
import forgi.threedee.model.coarse_grain as ftmc
import forgi.threedee.utilities.vector as ftuv
import forgi.utilities.commandline_utils as fuc
import forgi.threedee.classification.aminor as ftca
import forgi.threedee.utilities._dssr as ftud
import os
import numpy as np
import subprocess


directory = "../data/train_set/"

#dssr = ftud.DSSRAnnotation()

for i, file in enumerate([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]):
    cg = ftmc.CoarseGrainRNA.from_bg_file(directory + file) #("../../PDBs/3f2q/best_rmsd0.coord")

    dssr = subprocess.check_output(["x3dna-dssr", f"-i={directory+file}", "--json"])
    cg_d = dict(cg.coords)

    print(file)
    #A-minor interactions
    for pair in ftca.all_interactions(cg):
        print(pair)
    
    print(ftud.DSSRAnnotation(dssr=dssr ,cg=cg))
    #for elem in cg.sorted_element_iterator():
    #    print(elem, cg.connections(elem))
    
    #print(ftca.all_interactions(cg))

    if i == 5:
        break

5jb3_4_tRNA_4_rn37.cg
['h0' 's0']



Processing file '../data/train_set/5jb3_4_tRNA_4_rn37.cg'
no ATOM/HETATM records found!

Time used: 00:00:00:00


TypeError: dssr must be an string- or dict instance, not <class 'bytes'>